# Exciton Group Theory Analysis Tutorial

This tutorial demonstrates how to use the `ExcitonGroupTheory` class to analyze the symmetry properties of exciton states in crystalline materials.

## Overview

The `ExcitonGroupTheory` class provides comprehensive symmetry analysis including:

1. **Point Group Identification**: Automatic detection of crystallographic point groups
2. **Little Group Analysis**: Symmetry operations that preserve exciton momentum
3. **Irreducible Representation Decomposition**: Classification of exciton states
4. **Optical Activity Analysis**: Selection rules for optical transitions

## Theoretical Background

### Group Theory for Excitons

Exciton states transform according to the irreducible representations of the little group $G_k$ of the exciton momentum $\mathbf{k}$. The representation matrix for symmetry operation $R$ is:

$$D^{(n)}_R = \langle\psi_n(R\mathbf{k})| U(R) |\psi_n(\mathbf{k})\rangle$$

The character of this representation determines the irreducible representation:

$$\chi^{(n)}(R) = \text{Tr}[D^{(n)}_R]$$

### Selection Rules

Optical transitions follow symmetry selection rules:

- **Electric Dipole Transitions**: Allowed if $\Gamma_i \otimes \Gamma_f$ contains the representation of the dipole operator
- **Raman Scattering**: Allowed if the irrep is contained in the polarizability tensor
- **Infrared Absorption**: Allowed if the irrep transforms as a translation vector


## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from yambopy.optical_properties import ExcitonGroupTheory

# Set up plotting parameters
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## Example 1: hBN Monolayer Analysis

Let's analyze the exciton symmetries in hexagonal boron nitride (hBN), which has D₆ₕ point group symmetry.

In [ ]:
# Initialize ExcitonGroupTheory for hBN
# Adjust paths according to your calculation setup
egt = ExcitonGroupTheory(
    path='./hBN_calculation',
    BSE_dir='bse',
    LELPH_dir='lelph',
    bands_range=[6, 10],  # Valence and conduction bands around the gap
    read_symm_from_ns_db_file=True
)

print(f"System initialized with {len(egt.symm_mats)} symmetry operations")
print(f"Number of k-points in IBZ: {egt.nibz}")

### Analyze Exciton States at Γ Point

In [ ]:
# Perform group theory analysis at Γ point (iQ=1)
results = egt.analyze_exciton_symmetry(
    iQ=1,           # Γ point
    nstates=5,      # Analyze first 5 exciton states
    degen_thres=0.001  # Degeneracy threshold in eV
)

print("=" * 60)
print("EXCITON SYMMETRY ANALYSIS RESULTS")
print("=" * 60)
print(f"Q-point: {results['q_point']}")
print(f"Point group: {results['point_group_label']}")
print(f"Little group size: {len(results['little_group'])}")
print(f"Number of energy levels: {len(results['unique_energies'])}")

### Display Detailed Results

In [ ]:
print("\n" + "=" * 60)
print("ENERGY LEVELS AND SYMMETRIES")
print("=" * 60)
print(f"{'Level':<6} {'Energy (eV)':<12} {'Degeneracy':<11} {'Irrep':<15} {'Optical Activity'}")
print("-" * 80)

for i, (energy, degen, irrep, activity) in enumerate(zip(
    results['unique_energies'],
    results['degeneracies'],
    results['irrep_decomposition'],
    results['optical_activity']
)):
    # Determine optical activity status
    status = []
    if activity['electric_dipole_allowed']:
        status.append('Optical')
    if activity['raman_active']:
        status.append('Raman')
    if activity['ir_active']:
        status.append('IR')
    
    status_str = ', '.join(status) if status else 'Forbidden'
    
    print(f"{i+1:<6} {energy:<12.3f} {degen:<11} {irrep:<15} {status_str}")

### Visualize Energy Levels and Symmetries

In [ ]:
# Create energy level diagram
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 8))

# Energy level diagram
energies = results['unique_energies']
degeneracies = results['degeneracies']
irreps = results['irrep_decomposition']
activities = results['optical_activity']

# Plot energy levels
for i, (energy, degen, irrep, activity) in enumerate(zip(energies, degeneracies, irreps, activities)):
    # Color based on optical activity
    if activity['electric_dipole_allowed']:
        color = 'red'
        label = 'Optically Active' if i == 0 else ''
    else:
        color = 'blue'
        label = 'Optically Forbidden' if i == 0 else ''
    
    # Draw energy level
    ax1.hlines(energy, 0, 1, colors=color, linewidth=3, label=label)
    
    # Add labels
    ax1.text(1.1, energy, f'{irrep} (deg={degen})', 
             verticalalignment='center', fontsize=10)

ax1.set_xlim(-0.1, 2)
ax1.set_ylabel('Energy (eV)')
ax1.set_title('Exciton Energy Levels')
ax1.legend()
ax1.set_xticks([])

# Optical activity summary
activity_counts = {'Optical': 0, 'Raman': 0, 'IR': 0, 'Forbidden': 0}

for activity in activities:
    if activity['electric_dipole_allowed']:
        activity_counts['Optical'] += 1
    if activity['raman_active']:
        activity_counts['Raman'] += 1
    if activity['ir_active']:
        activity_counts['IR'] += 1
    if not any([activity['electric_dipole_allowed'], 
                activity['raman_active'], 
                activity['ir_active']]):
        activity_counts['Forbidden'] += 1

# Bar plot of activities
activities_list = list(activity_counts.keys())
counts = list(activity_counts.values())
colors = ['red', 'green', 'orange', 'gray']

bars = ax2.bar(activities_list, counts, color=colors, alpha=0.7)
ax2.set_ylabel('Number of States')
ax2.set_title('Optical Activity Summary')
ax2.set_ylim(0, max(counts) + 1)

# Add value labels on bars
for bar, count in zip(bars, counts):
    if count > 0:
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, 
                str(count), ha='center', va='bottom')

plt.tight_layout()
plt.show()

## Example 2: Analysis at Different k-points

Let's analyze how the symmetry changes at different k-points in the Brillouin zone.

In [ ]:
# Analyze multiple k-points
k_points_to_analyze = [1, 2, 3]  # Different k-points in the IBZ
k_point_results = {}

print("Analyzing exciton symmetries at different k-points...")
print("=" * 60)

for iQ in k_points_to_analyze:
    try:
        result = egt.analyze_exciton_symmetry(iQ=iQ, nstates=3, degen_thres=0.001)
        k_point_results[iQ] = result
        
        print(f"\nk-point {iQ}: {result['q_point']}")
        print(f"Point group: {result['point_group_label']}")
        print(f"Little group size: {len(result['little_group'])}")
        print(f"First exciton: {result['unique_energies'][0]:.3f} eV, {result['irrep_decomposition'][0]}")
        
    except Exception as e:
        print(f"Error analyzing k-point {iQ}: {e}")

## Example 3: Selection Rules Analysis

Let's examine the selection rules in detail for the first few exciton states.

In [ ]:
print("DETAILED SELECTION RULES ANALYSIS")
print("=" * 60)

# Focus on the first k-point results
main_results = results  # From the Γ point analysis above

for i, (energy, irrep, activity) in enumerate(zip(
    main_results['unique_energies'][:3],  # First 3 states
    main_results['irrep_decomposition'][:3],
    main_results['optical_activity'][:3]
)):
    print(f"\nExciton State {i+1}: {energy:.3f} eV")
    print(f"Irreducible Representation: {irrep}")
    print(f"Point Group: {main_results['point_group_label']}")
    print("-" * 40)
    
    # Selection rules
    print("Selection Rules:")
    print(f"  Electric Dipole Allowed: {activity['electric_dipole_allowed']}")
    print(f"  Raman Active: {activity['raman_active']}")
    print(f"  IR Active: {activity['ir_active']}")
    
    # Physical interpretation
    print("\nPhysical Interpretation:")
    if activity['electric_dipole_allowed']:
        print("  ✓ Observable in absorption/photoluminescence spectroscopy")
        print("  ✓ Can couple to light via electric dipole transitions")
    else:
        print("  ✗ Forbidden in electric dipole approximation")
        print("  ✗ Dark exciton (not directly observable in linear optics)")
    
    if activity['raman_active']:
        print("  ✓ Observable in Raman scattering experiments")
    
    if activity['ir_active']:
        print("  ✓ Observable in infrared spectroscopy")
    
    # Show detailed rules if available
    if activity['selection_rules']:
        print("\nDetailed Rules:")
        for j, rule in enumerate(activity['selection_rules']):
            if rule.get('notes'):
                for note in rule['notes']:
                    print(f"  • {note}")

## Example 4: Comparison with Experimental Data

Let's create a summary table that can be compared with experimental observations.

In [ ]:
import pandas as pd

# Create summary DataFrame
summary_data = []

for i, (energy, degen, irrep, activity) in enumerate(zip(
    results['unique_energies'],
    results['degeneracies'],
    results['irrep_decomposition'],
    results['optical_activity']
)):
    summary_data.append({
        'State': i + 1,
        'Energy (eV)': f"{energy:.3f}",
        'Degeneracy': degen,
        'Irrep': irrep,
        'Optical': '✓' if activity['electric_dipole_allowed'] else '✗',
        'Raman': '✓' if activity['raman_active'] else '✗',
        'IR': '✓' if activity['ir_active'] else '✗',
        'Character': 'Bright' if activity['electric_dipole_allowed'] else 'Dark'
    })

df = pd.DataFrame(summary_data)
print("SUMMARY TABLE FOR EXPERIMENTAL COMPARISON")
print("=" * 80)
print(df.to_string(index=False))

# Additional analysis
bright_states = sum(1 for activity in results['optical_activity'] 
                   if activity['electric_dipole_allowed'])
dark_states = len(results['optical_activity']) - bright_states

print(f"\nSUMMARY STATISTICS:")
print(f"Total states analyzed: {len(results['optical_activity'])}")
print(f"Bright excitons (optically active): {bright_states}")
print(f"Dark excitons (optically forbidden): {dark_states}")
print(f"Point group: {results['point_group_label']}")
print(f"Space group operations: {len(egt.symm_mats)}")

## Conclusion

This tutorial demonstrated the comprehensive capabilities of the `ExcitonGroupTheory` class:

1. **Automatic Point Group Detection**: The class correctly identifies the crystallographic point group using spglib
2. **Irreducible Representation Analysis**: Exciton states are decomposed into irreps of the little group
3. **Selection Rules**: Comprehensive analysis of optical, Raman, and IR activity
4. **Physical Interpretation**: Clear distinction between bright and dark excitons

### Key Insights for hBN:

- The system exhibits **C₆ₕ** or **D₆ₕ** point group symmetry
- Excitons transform according to **E-type irreps** (doubly degenerate)
- The lowest exciton is typically **optically active** (bright)
- Selection rules correctly predict experimental observability

### Applications:

This analysis is crucial for:
- **Interpreting optical spectra**: Understanding which peaks should be observable
- **Designing experiments**: Choosing appropriate spectroscopic techniques
- **Material characterization**: Identifying symmetry-breaking effects
- **Theoretical validation**: Comparing with experimental observations

The group theory analysis provides a fundamental understanding of exciton physics that complements and enhances traditional electronic structure calculations.